In [2]:
import pandas as pd
import unicodedata
import re
from collections import Counter, defaultdict
from itertools import islice
import json
from math import log2
from pathlib import Path
from math import floor

In [3]:
PATH = Path('.')

In [52]:
def normalize_title(title):
    s = re.sub(r'[^a-zA-Z0-9ñç% ]', ' ', unicodedata.normalize('NFKD', title.lower()).encode('ascii', 'ignore').decode("utf-8"))
    s = re.sub(r'[\d]+', "1", s)
    s = re.sub(r's |s$', ' ', s)
    s = re.sub(r' +', ' ', s)
    s = re.sub(r'(1 )+', '1 ', s)
    return re.sub(r'o |o$', 'a ', s).strip()

In [53]:
if not (PATH / 'train_prepro.csv').exists():
	df = pd.read_csv('train.csv')
	print("1")
	df['title'] = df.title.apply(normalize_title)
	print("2")
	df = df[~df.title.isna() & (df.title != 'nan') & (df.title != '')]
	df.to_csv(PATH / 'train_prepro.csv', index=False)

1
2


In [54]:
df = pd.read_csv('train_prepro.csv')

In [55]:
if not (PATH / 'important.json').exists():
	d = defaultdict(Counter)

	def process(r):
		ws = r['title'].split(' ')

		for w in ws:
			d[w][r['category']] += 1

	print("a")
	df.apply(process, axis=1)

	word_importance = dict()

	for w, c in d.items():
		importance = 0
		count = 0

		for x in c.most_common():
			if x[1] < 10:
				break
			count += 1	
			importance += x[1]

		if count > 0:	
			word_importance[w] = min(importance, 100000) / (count**2)



	with open(PATH / 'important.json', "w") as f:
		json.dump(word_importance, f, indent=4)

a


In [56]:
with open("important.json", "r") as f:
	word_importance = json.load(f)		

In [57]:
QTY = 60000

if not (PATH / f'vocab{QTY}.json').exists():

	cats = df['category'].nunique()
	total = 60000 - 15 * cats
	carry = 0
	important = set()
	ordered = []

	for index, rows in sorted(df.groupby('category'), key=lambda x: len(x[1]), reverse=True):
		available = 15 + floor(total * len(rows) / len(df)) + carry
		words = defaultdict(int)

		def process(r):
			ws = r.split(' ')
			weights = [word_importance.get(x, 0) for x in ws]
			total = sum(weights)

			for w, wei in zip(ws[0:len(ws)//2], weights):
				words[w] += wei / total if total > 0 else 1/len(ws)

		rows['title'].apply(process)

		for w in sorted(words.keys(), key = lambda x: words[x], reverse=True):
			if w not in important:
				important.add(w)
				ordered.append(w)
				available -= 1
				if available == 0:
					break

		carry = available


	for w in sorted(word_importance, key=lambda x: word_importance[x], reverse=True):
		if w not in important:
			important.add(w)
			ordered.append(w)
			if len(important) == QTY:
				break


	with open(PATH / f'vocab{QTY}.json', "w") as f:
		json.dump(ordered, f)
					

In [58]:
with open(PATH / f'vocab{QTY}.json', "r") as f:
	vocab = json.load(f)

In [59]:
len(vocab)

60000

In [60]:
vocab.index('de')

727

In [61]:
count = pd.Series(' '.join(df.title).split()).value_counts()

In [62]:
s = set(vocab)
from symspellpy.symspellpy import SymSpell  # import the module
sym_spell = SymSpell(2, 7)
for word in vocab:
    sym_spell.create_dictionary_entry(word, count[word])

In [67]:
a = 0
for word, c in count.iteritems():
    if word not in vocab and count[word] < 10:
        suggestions = sym_spell.lookup(word, 2)
        for suggestion in suggestions[0:1]:
            if suggestion.distance > 0:
                print(suggestion.term, suggestion.distance,
                              suggestion.count, word, count[word], count.get(suggestion.term, 0))
                a += 1  
    if a > 40:
        break

rusty 1 2403 rasty 9 2403
cocoa 1 359 cocopa 9 359
electrica 1 117983 electrica1 9 117983
bosch 1 62027 bossh 9 62027
cachoeira 1 573 cachoeira1 9 573
exata 1 314 exalta 9 314
intel 2 39685 injetel 9 39685
alti 1 238 altit 9 238
t1w 1 499 lt1w 9 499
kenwood 1 4374 kewood 9 4374
leiser 1 35 bleiser 9 35
morgan 2 879 mohuan 9 879
gira 2 32653 gojira 9 32653
tcs1f 1 80 tls1f 9 80
perola 1 9436 peroila 9 9436
prestan 1 21 prespan 9 21
fortune 1 319 fortunei 9 319
pms1 1 332 pmks1 9 332
kala 1 3135 kayla 9 3135
martor 1 58 martyr 9 58
usada 2 124667 kusama 9 124667
boligrafa 1 6461 boligraf 9 6461
microserva 2 15 microserie 9 15
fp1 1 1795 wfp1 9 1795
neumatica 1 21719 naumatica 9 21719
france 1 17382 franch 9 17382
aplicador 1 2711 aplicator 9 2711
maxiking 2 80 maxilink 9 80
dc1d1 1 12 dc1db1 9 12
vigueta 1 634 vegueta 9 634
expor 2 116 nexpod 9 116
arc 1 1680 arxc 9 1680
training 1 4789 traiding 9 4789
vp1tf 1 76 vk1tf 9 76
mascara 2 82735 malacara 9 82735
receptore 1 787 1receptore 9 78

In [12]:
from symspellpy.symspellpy import SymSpell  # import the module

In [13]:
sym_spell = SymSpell(2, 7)

In [15]:
for word in vocab:
    sym_spell.create_dictionary_entry(word, 1)

In [27]:
a = 0
for index, row in df.iterrows():
    suggestions = sym_spell.lookup_compound(row['title'],  2)
    for suggestion in suggestions[0:1]:
        if suggestion.distance > 0:
            print("{}, {}, {}, {}".format(suggestion.term, suggestion.distance,
                              suggestion.count, row['title']))
            a += 1
    if a > 20:
        break

hidrolavadora lavor one 1 bar 1w bomba aluminia itala, 1, 1, hidrolavadora lavor one 1 bar 1w bomba aluminia italia
kit tripe para celular ou camera motog 1 1m brinde nf e, 1, 1, kit tripe para celular ou camera fotog 1 1m brinde nf e
filtra ar bon anza 1 star1, 2, 1, filtra ar bonanza 1 sar1
condensador bosch vw fusca 1 1 de 1 a 1, 1, 1, condensador bosch vw fusca 1.1 de 1 a 1
rosaria contador de billete uv 1mg detecta falsa nueva, 1, 1, rosaria contador de billete uv mg detecta falsa nueva
caixa setor hidraulica ford focu 1 1 1, 1, 1, caixa setor hidraulica ford focu 1.1 1
porton de chapa 1 hoja mtr 1 1 sin marca, 1, 1, porton de chapa 1 hoja mtr 1.1 sin marca
vauen oxford 1 tabacca pipe cachimba madeira, 1, 1, vauen oxford 1 tobacca pipe cachimba madeira
faixa auta colante core surtida 1cm, 2, 1, faixa auta colante core sortida 1cm.
maquina para sublimar estampar moldex 1x1, 1, 1, maquina para sublimar estampar goldex 1x1
disfraz de general g levou para adulta toalla unica, 3, 1, di

In [26]:
SymSpell.lookup_compound??

In [29]:
vocab.index('vol.')

1677